In [1]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 77.6 MB/s eta 0:00:00


In [13]:
# Usage example
# Retrieve Nordpool prices of LT region and form a list
# Retrieval is from the legacy webservice providing info about all knwon hourly prices
import requests
import json

URL = "http://np-lt-day-ahead-pricing.azurewebsites.net/api/date/known_prices"
#URL = "http://np-lt-day-ahead-pricing.azurewebsites.net/api/date/tomorrow"
#URL = "http://np-lt-day-ahead-pricing.azurewebsites.net/api/date/2023-10-06"

np = requests.get(URL)
d=np.json()
data = d['data_current']

p = [] # create list of prices
h = [] # create list of hours
for k,v in data.items():
  p.append(v)
  h.append(k[:-6])
print(p)
print(h)


[17.49, 17.51, 17.5, 17.5, 17.6, 19.49, 20.6, 24.07, 23.95, 23.73, 23.95, 23.73, 16.32, 14.97, 14.33]
['2023-11-24 10', '2023-11-24 11', '2023-11-24 12', '2023-11-24 13', '2023-11-24 14', '2023-11-24 15', '2023-11-24 16', '2023-11-24 17', '2023-11-24 18', '2023-11-24 19', '2023-11-24 20', '2023-11-24 21', '2023-11-24 22', '2023-11-24 23', '2023-11-25 00']


In [14]:
import pulp

def optimize_power_consumption(battery_capacity=15, # in kwh
                               hourly_prices=[],    # a list of values, expressed in cents
                               hour_number=[],      # numbers of hours
                               initial_soc=0.7,
                               final_soc=0.5,
                               max_soc=1,
                               min_dod=0.2,
                               final_energy_value_per_kwh=12, # in cents
                               hourly_consumption=[],         # list of values, kwh
                               efficiency=1,
                               cost_of_cycle_kwh=5,           # in cents
                               max_allowed_grid_power=10,     # kw
                               max_charge_power=5,            # kw
                               max_discharge_power=5):        # kw
    # set variables
    n = len(hourly_prices)
    hours = range(n)
    initial_soc = battery_capacity * initial_soc
    final_soc = battery_capacity * final_soc

    # Create a linear programming problem
    problem = pulp.LpProblem("Power_Optimization", pulp.LpMinimize)

    # Define decision variables
    charge_rate = pulp.LpVariable.dicts("ChargeRate", hours, lowBound = 0, upBound= max_charge_power)
    discharge_rate = pulp.LpVariable.dicts("DischargeRate", hours, lowBound = -max_discharge_power, upBound=0)

    electricity_import = pulp.LpVariable.dicts("import", hours, lowBound=0, upBound=battery_capacity)
    soc = pulp.LpVariable.dicts("SOC", hours, lowBound=0, upBound=battery_capacity)

    # Define the objective function to minimize cost
    electricity_cost = pulp.lpSum(electricity_import[hour] * hourly_prices[hour] for hour in hours)
    amortization_cost = pulp.lpSum( (-discharge_rate[hour]) * cost_of_cycle_kwh for hour in hours)
    remaining_value = soc[n - 1] * efficiency * final_energy_value_per_kwh
    daily_cost = electricity_cost + amortization_cost - remaining_value

    problem += daily_cost

    # Define constraints
    for hour in hours:
      if hour == 0: init_soc = initial_soc
      else: init_soc = soc[hour -1]
      problem += soc[hour] == init_soc + charge_rate[hour] + discharge_rate[hour] # Conservation of charge
      problem += soc[hour] <= battery_capacity # battery soc does not exceed capacity
      problem += soc[hour] >= battery_capacity * min_dod # battery depth of discharge should not go lower than that
      problem += soc[hour] <= battery_capacity * max_soc # battery soc should not go above that
      problem += soc[n-1] >= final_soc # SOC has to be no less than target at the end of cycle
      problem += electricity_import[hour] == (hourly_consumption[hour] + charge_rate[hour] + discharge_rate[hour] * efficiency)
      problem += electricity_import[hour] >= 0
      problem += electricity_import[hour] <= max_allowed_grid_power

    # Solve the linear programming problem
    status = problem.solve()

    # Extract the results of the solution
    result = {
        "Hour": [],
        "HourlyPrices": hourly_prices,
        "HourlyConsumption": hourly_consumption,
        "ChargeRate": [],
        "DischargeRate": [],
        "ImportFromGrid": [],
        "Projected_SOC": [],
        "Command": [],
        "CMD": [],
        "OptimizationResult": {}
    }

    for hour in hours:
      result["Hour"].append(hour_number[hour])
      result["ChargeRate"].append(charge_rate[hour].varValue)
      result["DischargeRate"].append(discharge_rate[hour].varValue)
      result["ImportFromGrid"].append(electricity_import[hour].varValue)
      result["Projected_SOC"].append(soc[hour].varValue)
      command = charge_rate[hour].varValue + discharge_rate[hour].varValue
      result["Command"].append("Charge" if command > 0 else "Discharge" if command < 0 else "Idle")
      result["CMD"].append(str(command))

    # report consolidated optimization results
    result["OptimizationResult"]["Status"] = pulp.LpStatus[status]
    result["OptimizationResult"]["Cost w/o optimization"] = sum([hourly_prices[i] * hourly_consumption[i] for i in hours])/100


    consumed_kwh = sum([hourly_consumption[i]
        for i in hours])
    fromGrid_kwh = sum([result['ImportFromGrid'][i]
        for i in hours])

    charged_kwh = sum([result['ChargeRate'][i]
        for i in hours])
    avg_imported_kwh_price = sum([result['ImportFromGrid'][i] * hourly_prices[i]
        for i in hours]) / fromGrid_kwh / 100
    diff_init_final_SOC = initial_soc - result["Projected_SOC"][n-1]

    result["OptimizationResult"]["Cost after optimization"] = ((sum([result['ImportFromGrid'][i] * hourly_prices[i]  -
          result['DischargeRate'][i] * cost_of_cycle_kwh
        for i in hours])) + diff_init_final_SOC * avg_imported_kwh_price) / 100

    result["OptimizationResult"]["Consumed kwh"] = consumed_kwh
    result["OptimizationResult"]["Consumed from grid kwh"] = fromGrid_kwh
    result["OptimizationResult"]["Charged kwh to battery"] = charged_kwh
    result["OptimizationResult"]["Price of residual kwh"] = avg_imported_kwh_price * 100 * efficiency

    return result

# Example usage:
battery_capacity = 15  # kWh
hourly_prices = p
hour_number = h
initial_soc = 0.87  # %
final_soc= 0.5 # %
max_soc=1 # %
min_dod=0.2 # %
final_energy_value_per_kwh = 12 #  Estimate value of energy the first hour next day that helps us to decide what stateof charge should be left.
hourly_consumption = [1.6 for i in range(len(hourly_prices))]
cost_of_cycle_kwh = 5 # cents it costs per charge/discharge kwh (not cycle) of the battery (depreciacion costs)
max_allowed_grid_power = 22 # Max power privided by ESO
max_charge_power = 5 # maximum power in KW that battery can be charged with
max_discharge_power = 5 # maximum power in KW that battery can be discharged at
efficiency = 1

result = optimize_power_consumption(battery_capacity,
                                    hourly_prices,
                                    hour_number,
                                    initial_soc,
                                    final_soc,
                                    max_soc,
                                    min_dod,
                                    final_energy_value_per_kwh,
                                    hourly_consumption,
                                    efficiency,
                                    cost_of_cycle_kwh,
                                    max_allowed_grid_power,
                                    max_charge_power,
                                    max_discharge_power)


print("Report of commands")

title = ["HOUR", "PRICE", "CMD", "GRID", "SOC"]

print(f"{title[0]:>13} {title[1]:>8} {title[2]:>7} {title[3]:>7} {title[4]:>8}")
print(f"{initial_soc*battery_capacity:>47}")
for i in range(len(result['Hour'])):
  print(f"{result['Hour'][i]} | {result['HourlyPrices'][i]:6} | {result['CMD'][i]:>5} | {result['ImportFromGrid'][i]:5} | {result['Projected_SOC'][i]:>6}")

print()
print()
print("OPTIMIZATION RESULTS")
for k,v in result['OptimizationResult'].items():
  if isinstance(v, float):
    print(f"{k:>25}: {v:.2f}")
  else: print(f"{k:>25}: {v}")




Report of commands
         HOUR    PRICE     CMD    GRID      SOC
                                          13.05
2023-11-24 10 |  17.49 |   0.0 |   1.6 |  13.05
2023-11-24 11 |  17.51 |   0.0 |   1.6 |  13.05
2023-11-24 12 |   17.5 |   0.0 |   1.6 |  13.05
2023-11-24 13 |   17.5 |   0.0 |   1.6 |  13.05
2023-11-24 14 |   17.6 |   0.0 |   1.6 |  13.05
2023-11-24 15 |  19.49 | -0.45 |  1.15 |   12.6
2023-11-24 16 |   20.6 |  -1.6 |   0.0 |   11.0
2023-11-24 17 |  24.07 |  -1.6 |   0.0 |    9.4
2023-11-24 18 |  23.95 |  -1.6 |   0.0 |    7.8
2023-11-24 19 |  23.73 |  -1.6 |   0.0 |    6.2
2023-11-24 20 |  23.95 |  -1.6 |   0.0 |    4.6
2023-11-24 21 |  23.73 |  -1.6 |   0.0 |    3.0
2023-11-24 22 |  16.32 |   0.0 |   1.6 |    3.0
2023-11-24 23 |  14.97 |   0.0 |   1.6 |    3.0
2023-11-25 00 |  14.33 |   4.5 |   6.1 |    7.5


OPTIMIZATION RESULTS
                   Status: Optimal
    Cost w/o optimization: 4.68
  Cost after optimization: 3.51
             Consumed kwh: 24.00
   Consume